In [2]:
import pandas as pd
import numpy as np

In [3]:
metadata_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/CMS-1701-P%20Comment%20MetaData.csv"
data = pd.read_csv(metadata_url, usecols=range(0,36))[:469] #ignore last few columns and blank rows at end of csv 
data = data.rename(columns=lambda x: x.strip()) #strip whitespace from columns

In [4]:
data10 = data.fillna(0) #fill NaN with 0

In [5]:
section_cols = data10.columns[3:] 
data10[section_cols] = data10[section_cols].replace(["Y"], 1) #replace Y with 1 in approriate columns

In [6]:
section_cols


Index(['A2', 'A3', 'A4b', 'A4c', 'A5b', 'A5c', 'A5d', 'A6b', 'A6b.1', 'A6c',
       'A6d2', 'A6d3', 'A7', 'A7.1', 'A7.2', 'B2a', 'B2b', 'C2', 'C3a', 'C3b',
       'C3b.1', 'D2', 'D3b', 'D3c', 'D3d', 'D4', 'E2', 'E3', 'E4', 'E5', 'E6',
       'E7', 'F'],
      dtype='object')

In [38]:
data11 = data10.copy()

In [69]:
section_cols1  = data11.columns[3:] 

In [71]:
data11[section_cols1] = np.where((data11[section_cols1]  != 1),0,data11[section_cols1] )
data11



,Name,Organization Name / Submitter name,Submitter State,A2,A3,A4b,A4c,A5b,A5c,A5d,...,D3c,D3d,D4,E2,E3,E4,E5,E6,E7,F
0,CMS-2018-0101-DRAFT-0002-08_23_2018-05_17_PM,Erick Meleher,NC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CMS-2018-0101-DRAFT-0003-08_23_2018-05_18_PM,Mayank Shah,IL,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,CMS-2018-0101-DRAFT-0004-08_24_2018-01_16_PM,Mayank Shah,IL,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CMS-2018-0101-DRAFT-0005-08_29_2018-10_05_AM,Morey Menacker,NJ,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CMS-2018-0101-DRAFT-0006-08_30_2018-09_14_AM,Todd Rapoza,MA,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,CMS-2018-0101-DRAFT-0466-10_17_2018-04_53_AM,Liberty ACO,TX,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
465,CMS-2018-0101-DRAFT-0467-10_17_2018-04_53_AM,Washington State Hospital Association,WA,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
466,CMS-2018-0101-DRAFT-0468-10_17_2018-05_02_AM,Think Whole Person Healthcare,NE,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
467,CMS-2018-0101-DRAFT-0469-10_17_2018-05_02_AM,Palm Beach Accountable Care Organization,FL,0,1,0,1,1,1,0,...,1,0,0,0,1,1,0,1,0,0


In [73]:
# Combining columns for index matching: (A6b, A6b.1, = A6b),  (C3b, C3b.1'= C3b) ('A7', 'A7.1', 'A7.2', = A7b, a7c),  (F = F2, F3)


data11['A6b'] = (data11['A6b'] + data11['A6b.1'])
data11['A6b'] = data11['A6b'].replace(2,1)
data11['A6b'].value_counts()

0    448
1     21
Name: A6b, dtype: int64

In [75]:
data11['C3b'] = (data11['C3b'] + data11['C3b.1'])
data11['C3b'] = data11['C3b'].replace(2,1)
data11['C3b'].value_counts()

0    413
1     56
Name: C3b, dtype: int64

In [76]:
data11['A7'] = (data11['A7'] + data11['A7.1'] + data11['A7.2'])
data11['A7'] = data11['A7'].replace(2,1)
data11['A7'] = data11['A7'].replace(3,1)
data11['A7'].value_counts()

0    418
1     51
Name: A7, dtype: int64

In [ ]:
data11 = data11.drop(['A6b.1', 'C3b.1', 'A7.1', 'A7.2'], axis = 1)
data11

In [86]:
section_cols1  = data11.columns[3:] 

In [89]:
truth_dict = {} # initiate dict
for i, name in enumerate(data11.Name): # iterate through Comment Names
    dict1 = {col:data11[col].iloc[i] for col in section_cols1} # locate this row's values for each column
    list1 = [key for key,value in dict1.items() if value==1] # create list of matching sections for this row
    name1 = str(name).split('DRAFT-0')[1].split('-')[0] # format Comment Name
    truth_dict[name1] = list1 # add Comment Name and list of matching sections to dict

In [90]:
truth_dict

{'002': ['D2'],
 '003': ['D2', 'D3c', 'E5'],
 '004': ['A7'],
 '005': ['A5b'],
 '006': ['F'],
 '007': ['E3'],
 '008': ['C3b'],
 '009': ['A5b'],
 '010': ['A3', 'A5b'],
 '011': ['A3', 'A5b'],
 '012': ['A2', 'A4c', 'A5b', 'A7', 'C2', 'D2', 'D3b', 'E5'],
 '013': ['A2', 'A3', 'A5b', 'C2', 'D2', 'D3b', 'D3c', 'D3d'],
 '014': ['A2', 'A3', 'A4c', 'C3a', 'D2', 'D3b', 'D3c', 'E6'],
 '015': ['E2'],
 '016': ['E5'],
 '017': ['E2'],
 '018': ['E2'],
 '019': ['E2'],
 '020': ['E2'],
 '021': ['E2'],
 '022': ['E2'],
 '023': ['E2'],
 '024': ['E2'],
 '025': ['E2'],
 '026': ['E2'],
 '027': ['E2'],
 '028': ['E2'],
 '029': ['E2'],
 '030': ['E2'],
 '031': ['E2'],
 '032': ['E2'],
 '033': ['E2'],
 '034': ['E2'],
 '035': ['E2'],
 '036': ['E2'],
 '037': ['E2'],
 '038': ['E2'],
 '039': ['E2'],
 '040': ['E2'],
 '041': ['E2'],
 '042': ['A5c'],
 '043': ['E2'],
 '044': ['E2'],
 '045': ['E2'],
 '046': ['E2'],
 '047': ['E2'],
 '048': ['E2'],
 '049': ['E2'],
 '050': ['E2'],
 '051': ['E2'],
 '052': ['E2'],
 '053': ['E2'],
 

In [79]:
num_matches = {key: len(value) for key, value in truth_dict.items()}

In [53]:
#num_matches